# Using Selenium and Requests: _Ahimasa Collective, Mimco, and Reformation_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Ahimasa Collective and Mimco.First, we will use selenium to get everylink for the bags, and then use Beautiful Soup to get the correct picture,price, product link, and description.

Some great sources used to help in this: 
- [Selenium Website for Locating Elements](https://selenium-python.readthedocs.io/locating-elements.html)
- [Stackover flow to Save Images](https://stackoverflow.com/questions/18497840/beautifulsoup-how-to-open-images-and-download-them)
- [Get all image src's with specific class](https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract)


Our steps are as follows: 
1. [first]()
2. [second]()
3. [third]()
4. [fourth]() 

#### Import your needed Libraries:

In [1]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO


#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Ahimasa Collective and Mimco_.

In [3]:
ahimasa_co ='https://www.theahimsacollective.com/collections/all'
mimco = ['https://www.mimco-accessories.com/shop/bags/backpacks','https://www.mimco-accessories.com/shop/bags/belt-bags',
        'https://www.mimco-accessories.com/shop/bags/hip-bags','https://www.mimco-accessories.com/shop/bags/shoulder-bags#catpage=3',
        'https://www.mimco-accessories.com/shop/bags/tote-bags#catpage=3']

In [4]:
page = requests.get(ahimasa_co)
soup = BeautifulSoup(page.content, 'lxml')

In [24]:
class_ = 'grid-view-item__image-wrapper js'
divs = soup.findAll('div', {'class':class_})
images = ['https:'+ x.img['src'] for x in divs]
infos = soup.findAll('span', {'class':'product-price__price'})
prices = [x.text for x in infos]
name_div = soup.findAll('div',{'class':'h4 grid-view-item__title'})
names = [x.text for x in name_div]

In [26]:
ahimasa_dict = {'description':names,'price':prices,'link':images}
ahimasa_df = pd.DataFrame(ahimasa_dict)

In [77]:
keep1 = list(ahimasa_df.description.values[3:9])
keep2 = list(ahimasa_df.description.values[15:17])
keep3 = ahimasa_df.description.values[-1]
# keep = np.concatenate([keep1,keep2,keep3])

In [78]:
keep1.extend(keep2)
keep1.append(keep3)
keep1

In [95]:
ind = []
for i,val in enumerate(ahimasa_df.description):
    if val in keep1:
        ind.append(i)

In [94]:
ahimasa_df = ahimasa_df.loc[ind, : ]

In [112]:
s = 'raven lily'
s
''.join([i[0] for i in s.split(' ')])

'rl'

In [123]:
c = 0
def save_src_image_apply(company,short,url):
    start_time = time.time()
    time.sleep(1)
    
    try:
        global c
        c += 1
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        image_save_loc = company+short+'_'+str(c)+'.jpg'
        img.save(image_save_loc, "JPEG")
        print("Saved "+short+"_{}".format(c)+".jpg")
        return short+"_"+ str(c)+".jpg"
    except Exception as e:
        return e

In [121]:
ahimasa_df['img'] = ahimasa_df.link.apply(lambda x: save_src_image_apply('ahimasa_co/','a_co',x))

Saved a_co_1.jpg
1
Saved a_co_2.jpg
2
Saved a_co_3.jpg
3
Saved a_co_4.jpg
4
Saved a_co_5.jpg
5
Saved a_co_6.jpg
6
Saved a_co_7.jpg
7
Saved a_co_8.jpg
8
Saved a_co_9.jpg
9


In [246]:
ahimasa_df.to_csv('data/ahimasa_co.csv')

##### Mimco:

In [126]:
link = 'https://www.mimco-accessories.com/shop/bags/tote-bags#catpage=3'

page = requests.get(link)
soup = BeautifulSoup(page.content, 'lxml')

bags = soup.findAll('a', {'class':'product_link'})
images = ['https://www.mimco-accessories.com'+ x.img['src'] for x in bags]
names = [x['title'].lower() for x in bags]
prices_tag = soup.findAll('span', {'class':'value'})
prices = [x.text for x in prices_tag]

In [ ]:
url = 'https://www.mimco-accessories.com/shop/bags/tote-bags#catpage=3'

def products_to_DF(links):
    images = []
    names = []
    prices = []
    
    for link in links: 
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'lxml')
        
        bags = soup.findAll('figure', {'class':'product-list-item-thumbnail'})
        images.extend(['https:'+ x.img['src'] for x in bags])
        names.extend([x.img['alt'] for x in bags])
        prices_tag = soup.findAll('p', {'class':'product-list-item-price'})
        prices.extend([x.text for x in prices_tag])
    
    return pd.DataFrame({'description':names,'link':images,'price':prices})

In [234]:
def scrape_mimco(urls):
    
    all_ = []
    links = []
    desc = []
    prices = []
    
    for url in urls:

        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.get(url)

        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            elements = (browser.find_elements_by_class_name(name= 'product_link'))
            all_.extend(elements)
            links_start = [e.get_attribute('innerHTML') for e in elements]
            links_2 = [re.search('.*?\" src\=\"(.*)\" data\-mouseimage', link) for link in links_start]
            links.extend(['https://www.mimco-accessories.com'+link.group(1) for link in links_2])
            desc.extend([e.get_attribute('title') for e in elements])
            elements2 = (browser.find_elements_by_class_name(name= 'value'))
            prices.extend([e.text for e in elements2])
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

            if lastCount==lenOfPage:
                match=True
        browser.close()
    
    df = pd.DataFrame({'description':desc,'link':links,'price':prices}).drop_duplicates()
    return df.reset_index(drop=True)


##### Create a dataframe using the function above:

In [235]:
mimco_df = scrape_mimco(['https://www.mimco-accessories.com/shop/bags/tote-bags',
                         'https://www.mimco-accessories.com/shop/bags/shoulder-bags',
                         'https://www.mimco-accessories.com/shop/bags/belt-bags',
                         'https://www.mimco-accessories.com/shop/bags/hip-bags'])


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver


In [237]:
mimco_df.head(10)

,description,link,price
0,SENTIMENT MINI TOTE,https://www.mimco-accessories.com/ProductImage...,293.78
1,FLIP SIDE TOTE,https://www.mimco-accessories.com/ProductImage...,331.33
2,MODERNIST TOTE,https://www.mimco-accessories.com/ProductImage...,367.41
3,FLIP SIDE TOTE,https://www.mimco-accessories.com/ProductImage...,331.33
4,SUBLIME TOTE,https://www.mimco-accessories.com/ProductImage...,293.78
5,SENTIMENT MINI TOTE,https://www.mimco-accessories.com/ProductImage...,293.78
6,Flip Side Tote,https://www.mimco-accessories.com/ProductImage...,331.33
7,HYDRO TOTE,https://www.mimco-accessories.com/ProductImage...,183.34
8,Amorphous Mini Tote,https://www.mimco-accessories.com/ProductImage...,331.33
9,SUBLIME TOTE,https://www.mimco-accessories.com/ProductImage...,293.78


### Download Mimco images:

Download images using the __*save_src_image_apply*__ function. Create a new column in the df using apply.

In [240]:
c = 0
def save_src_image_apply(company,short,url):
    start_time = time.time()
    time.sleep(1)
    
    try:
        global c
        c += 1
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        image_save_loc = company+short+'_'+str(c)+'.png'
        img.save(image_save_loc, "PNG")
        print("Saved "+short+"_{}".format(c)+".png")
        return short+"_"+ str(c)+".png"
    except Exception as e:
        return e

In [241]:
mimco_df['img'] = mimco_df.link.apply(lambda x: save_src_image_apply('mimco/','mimco',x))

Saved mimco_1.png
Saved mimco_2.png
Saved mimco_3.png
Saved mimco_4.png
Saved mimco_5.png
Saved mimco_6.png
Saved mimco_7.png
Saved mimco_8.png
Saved mimco_9.png
Saved mimco_10.png
Saved mimco_11.png
Saved mimco_12.png
Saved mimco_13.png
Saved mimco_14.png
Saved mimco_15.png
Saved mimco_16.png
Saved mimco_17.png
Saved mimco_18.png
Saved mimco_19.png
Saved mimco_20.png
Saved mimco_21.png
Saved mimco_22.png
Saved mimco_23.png
Saved mimco_24.png
Saved mimco_25.png
Saved mimco_26.png
Saved mimco_27.png
Saved mimco_28.png
Saved mimco_29.png
Saved mimco_30.png
Saved mimco_31.png
Saved mimco_32.png
Saved mimco_33.png
Saved mimco_34.png
Saved mimco_35.png
Saved mimco_36.png
Saved mimco_37.png
Saved mimco_38.png
Saved mimco_39.png
Saved mimco_40.png
Saved mimco_41.png
Saved mimco_42.png
Saved mimco_43.png
Saved mimco_44.png
Saved mimco_45.png
Saved mimco_46.png
Saved mimco_47.png
Saved mimco_48.png
Saved mimco_49.png
Saved mimco_50.png
Saved mimco_51.png
Saved mimco_52.png
Saved mimco_53.png
Sa

In [245]:
mimco_df.to_csv('data/mimco.csv')

### Reformation

This process will be a little more manual due to the few products, and the quality of the pictures.
We will manually create a dataframe with all the correct columns.

In [242]:
links = ['https://res.cloudinary.com/reformation/image/upload/q_auto:eco/c_scale,w_auto:breakpoints_100_2560_9_20:862/v1/prod/product_images/canvas-tote/comic/5c8027039dd3aa5a5ac64ca8/original.jpg',
         'https://res.cloudinary.com/reformation/image/upload/q_auto:eco/c_scale,w_auto:breakpoints_100_2560_9_20:862/v1/prod/product_images/canvas-tote/black/5c8027059dd3aa5a5ac64caa/original.jpg',
         'https://res.cloudinary.com/reformation/image/upload/q_auto:eco/c_scale,w_auto:breakpoints_100_2560_9_20:862/v1/prod/product_images/canvas-tote/natural/5c8027049dd3aa5a5ac64ca9/original.jpg']
desc = ['Beach Tote - Comic','Beach Tote - Black','Beach Tote - Natural']
prices = [38.0]*3
imgs = ['ref_1.jpg','ref_2.jpg','ref_3.jpg']

ref_df = pd.DataFrame({'link':links,'description':desc,'price':prices,'img':imgs})

In [244]:
ref_df.to_csv('data/reformation.csv')